#### Goal
This notebook will be dedicated to extracting the embeddings out of the video data in order to train a OPT. LETS FUCKING GO BOY



In [ ]:
!pip install remotezip

In [ ]:
!pip install tf-models-official

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.1 MB/s eta 0:00:00
  Created wheel for seqeva

In [ ]:
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import einops
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers

In [ ]:
import pandas as pd
import numpy as np
import pickle
import tqdm
import random
import pathlib
import itertools
import collections

import cv2
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
# Import the MoViNet model from TensorFlow Models (tf-models-official) for the MoViNet model
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model

### Importing the Files

In [ ]:
def import_frames(file_path=None):
  with open(file_path, "rb") as file:
    data = pickle.load(file)
    return data

In [ ]:
t = [len(train_data[0]) for data in train_data]
sum(t)

NameError: name 'train_data' is not defined

In [ ]:
# train and test set
train_data_ = []
for i in range(9):
  try:
    train_data_.append(import_frames(file_path = f"week_{i+1}_no_lines.pkl"))
  except:
    continue

In [ ]:
# we first need to flatten the data
# Flatten train_data into a single list of (input_sequence, output) pairs
flattened_data = [(example[0], example[1]) for dataset in train_data_ for example in dataset]

In [ ]:
split_size = int(0.8 * len(flattened_data))  # 80% train, 20% test
train_data = flattened_data[:split_size]
test_data = flattened_data[split_size:]

In [ ]:
# we need to resize and normalize the image
img_height = 100
img_width = 200
def preprocess_image(image_sequence):
  # resize the image to make it much smaller
  preprocess_sequence = [
      tf.image.resize(tf.expand_dims(image, axis=-1), (img_height, img_width)) / 255.0 for image in image_sequence
  ]
  return tf.stack(preprocess_sequence)

In [ ]:
# we will also have to pad the maximum length in the batch to account for diff frame
def padding_batches(input_sequence, output):
    max_length = 30  # Replace this with the maximum expected sequence length in your data
    padded_sequence = tf.pad(
        input_sequence,
        paddings=[[0, max_length - tf.shape(input_sequence)[0]], [0, 0], [0, 0], [0,0]],
        mode="CONSTANT",
        constant_values=0
    )
    return padded_sequence, output


In [ ]:
# take everything and wrap this into a dataset
def train_generator():
    for input_sequence, output in train_data:
        padded_sequence, label = padding_batches(preprocess_image(input_sequence), output)
        yield padded_sequence, label


def test_generator():
    for input_sequence, output in test_data:
        padded_sequence, label = padding_batches(preprocess_image(input_sequence), output)
        yield padded_sequence, label

train_dataset = tf.data.Dataset.from_generator(
    train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_height, img_width, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32),
    )
)

test_dataset = tf.data.Dataset.from_generator(
    test_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_height, img_width, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32),
    )
)


# pad our sequences in a batch
BATCH_SIZE = 32

train_dataset = train_dataset.padded_batch(
    BATCH_SIZE,
    padded_shapes = (
        [None, img_height, img_width, 1],
        []
    )
)

test_dataset = test_dataset.padded_batch(
    BATCH_SIZE,
    padded_shapes = (
        [None, img_height, img_width, 1],
        []
    )
)



In [ ]:
class Conv2Plus1D(keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding):
    """
      A sequence of convolutional layers that first apply the convolution operation over the
      spatial dimensions, and then the temporal dimension.
    """
    super().__init__()
    self.seq = keras.Sequential([
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]),
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(kernel_size[0], 1, 1),
                      padding=padding)
        ])

  def call(self, x):
    return self.seq(x)

In [ ]:
from tensorflow.keras import layers, models

from tensorflow.keras import layers, models

# Assuming HEIGHT and WIDTH are defined as per your input dimensions
input_shape = (None, 20, 100, 200, 1)  # Input shape (None, frames, height, width, channels)
input = layers.Input(shape=(input_shape[1:]))

# First convolutional layer with batch normalization and ReLU activation
x = Conv2Plus1D(filters=2, kernel_size=(7, 20, 5), padding='valid')(input)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

# Block 1: Convolutional layer followed by resizing
x = Conv2Plus1D(filters=4, kernel_size=(7, 10, 5), padding='valid')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)



# Global average pooling to reduce the feature map to a single vector
x = layers.GlobalAveragePooling3D()(x)

# Flatten the output for the fully connected layer
x = layers.Flatten()(x)

# Final regression output (single unit)
x = layers.Dense(100)(x)
x = layers.ReLU()(x)

x = layers.Dense(50)(x)
x = layers.ReLU()(x)

x = layers.Dense(10)(x)

# add a relu
x = layers.ReLU()(x)

x = layers.Dense(5)(x)
x = layers.ReLU()(x)

x = layers.Dense(1)(x)

# Create model
model = models.Model(inputs=input, outputs=x)

# Compile the model

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=optimizer,
    metrics=['accuracy']
)

# Display the model summary
model.summary()


Model: "functional_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_24 (InputLayer)          │ (None, 20, 100, 200, 1)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2_plus1d_16 (Conv2Plus1D)        │ (None, 14, 81, 196, 2)      │             232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ (None, 14, 81, 196, 2)      │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_29 (ReLU)                      │ (None, 14, 81, 196, 2)      │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2_plus1d_17 (Conv2Plus1D)        │ (None, 8, 72, 192, 4)       │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ (None, 8, 72, 192, 4)       │              16 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_30 (ReLU)                      │ (None, 8, 72, 192, 4)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling3d_7           │ (None, 4)                   │               0 │
│ (GlobalAveragePooling3D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ (None, 4)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 100)                 │             500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_31 (ReLU)                      │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_32 (ReLU)                      │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 10)                  │             510 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_33 (ReLU)                      │ (None, 10)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 5)                   │              55 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_34 (ReLU)                      │ (None, 5)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 1)                   │               6 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,897 (26.94 KB)

 Trainable params: 6,885 (26.89 KB)

 Non-trainable params: 12 (48.00 B)

In [ ]:
history = model.fit(
    train_dataset,  # The dataset you created
    epochs=15,       # Number of epochs to train
    validation_data=test_dataset,  # Use test dataset for validation
    verbose=1
)




Epoch 1/15
     46/Unknown 81s 851ms/step - accuracy: 0.0945 - loss: 83.1151

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


46/46 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.0939 - loss: 82.8594 - val_accuracy: 0.0609 - val_loss: 70.4395
Epoch 2/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 888ms/step - accuracy: 0.0578 - loss: 67.6740 - val_accuracy: 0.0609 - val_loss: 68.7422
Epoch 3/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 45s 976ms/step - accuracy: 0.0578 - loss: 66.6903 - val_accuracy: 0.0609 - val_loss: 68.2822
Epoch 4/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 78s 889ms/step - accuracy: 0.0578 - loss: 66.2233 - val_accuracy: 0.0609 - val_loss: 68.6439
Epoch 5/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 900ms/step - accuracy: 0.0578 - loss: 66.3055 - val_accuracy: 0.0609 - val_loss: 693.3315
Epoch 6/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 82s 893ms/step - accuracy: 0.0578 - loss: 66.1202 - val_accuracy: 0.0609 - val_loss: 73.1386
Epoch 7/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 40s 879ms/step - accuracy: 0.0578 - loss: 65.7696 - val_accuracy: 0.0609 - val_loss: 68.1997
Epoch 8/15
46/46 ━━━━━━━━━━━━━━━━━━━━ 41s 887ms/step - accuracy: 0.0578 - loss: 65.4835 - val_ac

In [ ]:
history.history['loss']

[96.2010726928711,
 91.57322692871094,
 88.73553466796875,
 85.61486053466797,
 82.04069519042969,
 77.84407043457031,
 73.2854232788086,
 68.5335922241211,
 65.62260437011719,
 65.06866455078125]

In [ ]:
history.history['val_loss']

[95.31227111816406,
 93.16265106201172,
 90.5123291015625,
 87.50639343261719,
 84.05874633789062,
 79.92640686035156,
 75.29935455322266,
 72.28152465820312,
 67.17628479003906,
 65.43704986572266]

In [ ]:
# Assuming your original model is already trained
# extract embeddings for the model
embedding_model = keras.Model(inputs=model.input, outputs=model.get_layer('dense_29').output)

# Get embeddings for the training and test sets
train_embeddings = embedding_model.predict(train_dataset)
test_embeddings = embedding_model.predict(test_dataset)

# Optionally, save embeddings
# np.save('train_embeddings.npy', train_embeddings)
# np.save('test_embeddings.npy', test_embeddings)


36/36 ━━━━━━━━━━━━━━━━━━━━ 19s 509ms/step


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 536ms/step


In [ ]:
train_embeddings.shape

(1140, 10)

In [ ]:
test_embeddings.shape

(286, 10)

In [ ]:
ids = []
for i in range(8):
  try:
    ids.append(import_frames(file_path = f"good_girl_{i+1}.pkl"))
  except:
    continue

In [ ]:
len(train_data)

1140

In [ ]:
[len(idss) for idss in ids]

[236, 105, 359, 371, 133, 113, 109]

In [ ]:
[len(data) for data in train_data_]

[236, 105, 359, 371, 133, 113, 109]

In [ ]:
flat_ids = [item for batch in ids for item in batch]
len(flat_ids)

1426

In [ ]:
full_embeddings = np.concatenate(
    (train_embeddings, test_embeddings),
    axis=0)

In [ ]:
df_ids = pd.DataFrame([{"game_id": game_id, "play_id": play_id} for game_id, play_id in flat_ids])

In [ ]:
df_embeddings = pd.DataFrame(full_embeddings, columns=[f"emb{i+1}" for i in range(full_embeddings.shape[1])])


In [ ]:
df_embeddings

,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,emb9,emb10
0,0.638034,0.028732,0.386127,-0.498500,1.383350,-0.747871,-0.490596,0.729719,-0.308404,0.236588
1,0.209874,-0.080251,0.324387,-0.229344,0.811116,-0.284877,-0.062086,0.502232,-0.038244,-0.056882
2,0.334418,-0.050700,0.340614,-0.304905,0.972251,-0.416566,-0.187807,0.565736,-0.116735,0.026979
3,0.334044,-0.050846,0.340536,-0.304610,0.971612,-0.416081,-0.187429,0.565443,-0.116457,0.026655
4,0.334425,-0.050712,0.340605,-0.304892,0.972238,-0.416564,-0.187825,0.565737,-0.116745,0.026985
...,...,...,...,...,...,...,...,...,...,...
1421,-0.047799,-0.113828,0.303247,-0.104842,0.542509,-0.048374,0.199670,0.408788,0.107977,-0.204012
1422,0.209283,-0.080454,0.324257,-0.228904,0.810218,-0.284181,-0.061533,0.501882,-0.037887,-0.057300
1423,-0.047810,-0.113822,0.303231,-0.104839,0.542502,-0.048364,0.199662,0.408793,0.107955,-0.204005
1424,0.091826,-0.107235,0.309873,-0.159042,0.660973,-0.161517,0.057629,0.443341,0.036148,-0.135646


In [ ]:
full_df = pd.concat([df_ids, df_embeddings], axis=1)


In [ ]:
full_df

,game_id,play_id,emb1,emb2,emb3,emb4,emb5,emb6,emb7,emb8,emb9,emb10
0,2022090800,56,0.638034,0.028732,0.386127,-0.498500,1.383350,-0.747871,-0.490596,0.729719,-0.308404,0.236588
1,2022090800,80,0.209874,-0.080251,0.324387,-0.229344,0.811116,-0.284877,-0.062086,0.502232,-0.038244,-0.056882
2,2022090800,101,0.334418,-0.050700,0.340614,-0.304905,0.972251,-0.416566,-0.187807,0.565736,-0.116735,0.026979
3,2022090800,122,0.334044,-0.050846,0.340536,-0.304610,0.971612,-0.416081,-0.187429,0.565443,-0.116457,0.026655
4,2022090800,167,0.334425,-0.050712,0.340605,-0.304892,0.972238,-0.416564,-0.187825,0.565737,-0.116745,0.026985
...,...,...,...,...,...,...,...,...,...,...,...,...
1421,2022103012,3671,-0.047799,-0.113828,0.303247,-0.104842,0.542509,-0.048374,0.199670,0.408788,0.107977,-0.204012
1422,2022103012,3693,0.209283,-0.080454,0.324257,-0.228904,0.810218,-0.284181,-0.061533,0.501882,-0.037887,-0.057300
1423,2022103012,3717,-0.047810,-0.113822,0.303231,-0.104839,0.542502,-0.048364,0.199662,0.408793,0.107955,-0.204005
1424,2022103012,3739,0.091826,-0.107235,0.309873,-0.159042,0.660973,-0.161517,0.057629,0.443341,0.036148,-0.135646
